https://github.com/pytorch/examples/blob/a38cbfc6f817d9015fc67a6309d3a0be9ff94ab6/mnist/main.py

In [85]:
%%bash
pip install torch

In [86]:
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR

In [87]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)
        self._i:int=0

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)


        self._i=self._i+1
        if self._i % 100==0:
          pass
          # print(f"i@forward={self._i}")
          # print(f"[LOGG@forward] type(x)={type(x)}, x.shape={x.shape}; type(output)={type(output)}, output.shape={output.shape}")
          # [LOGG@forward] type(x)=<class 'torch.Tensor'>, x.shape=torch.Size([64, 10]); type(output)=<class 'torch.Tensor'>, output.shape=torch.Size([64, 10])

        return output

In [88]:
log_interval=40

def train( model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        # print(f"[LOGG@train] type(data)={type(data)}, data.shape={data.shape}; type(target)={type(target)}, target.shape={target.shape}")
        # [LOGG@train] type(data)=<class 'torch.Tensor'>, data.shape=torch.Size([64, 1, 28, 28]); type(target)=<class 'torch.Tensor'>, target.shape=torch.Size([64])
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


In [89]:


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            # print(f"[LOGG@test] type(data)={type(data)}, data.shape={data.shape};  type(target)={type(target)}, target.shape={target.shape}")
            # [LOGG@test] type(data)=<class 'torch.Tensor'>, data.shape=torch.Size([1000, 1, 28, 28]);  type(target)=<class 'torch.Tensor'>, target.shape=torch.Size([1000])
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))


In [90]:
device = torch.device("cpu")
batch_size=64
test_batch_size=1000
epochs=2
lr=1.0
gamma=0.7

In [91]:
train_kwargs = {'batch_size': batch_size}
test_kwargs = {'batch_size': test_batch_size}


transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
    ])
dataset1 = datasets.MNIST('../data', train=True, download=True,
                    transform=transform)
dataset2 = datasets.MNIST('../data', train=False,
                    transform=transform)
# print(f"[LOGG@main] type(dataset1)={type(dataset1)} ;  type(dataset2)={type(dataset2)} ")
#[LOGG@main] type(dataset1)=<class 'torchvision.datasets.mnist.MNIST'> ;  type(dataset2)=<class 'torchvision.datasets.mnist.MNIST'>
#https://github.com/pytorch/vision/blob/v0.18.1/torchvision/datasets/mnist.py
# print(f"[LOGG@main] type(dataset1)={type(dataset1)}, dataset1.train_data.shape={dataset1.train_data.shape}, dataset1.train_labels.shape={dataset1.train_labels.shape}; dataset1.test_data.shape={dataset1.test_data.shape}, dataset1.test_labels.shape={dataset1.test_labels.shape}")
#[LOGG@main] type(dataset1)=<class 'torchvision.datasets.mnist.MNIST'>, dataset1.train_data.shape=torch.Size([60000, 28, 28]), dataset1.train_labels.shape=torch.Size([60000]); dataset1.test_data.shape=torch.Size([60000, 28, 28]), dataset1.test_labels.shape=torch.Size([60000])
# print(f"[LOGG@main] type(dataset2)={type(dataset2)}, dataset2.train_data.shape={dataset2.train_data.shape}, dataset2.train_labels.shape={dataset2.train_labels.shape}; dataset2.test_data.shape={dataset2.test_data.shape}, dataset2.test_labels.shape={dataset2.test_labels.shape}")
# [LOGG@main] type(dataset2)=<class 'torchvision.datasets.mnist.MNIST'>, dataset2.train_data.shape=torch.Size([10000, 28, 28]), dataset2.train_labels.shape=torch.Size([10000]); dataset2.test_data.shape=torch.Size([10000, 28, 28]), dataset2.test_labels.shape=torch.Size([10000])
train_loader = torch.utils.data.DataLoader(dataset1,**train_kwargs)
test_loader = torch.utils.data.DataLoader(dataset2, **test_kwargs)

model = Net().to(device)
optimizer = optim.Adadelta(model.parameters(), lr=lr)

scheduler = StepLR(optimizer, step_size=1, gamma=gamma)
for epoch in range(1, epochs + 1):
    train( model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)
    scheduler.step()

torch.save(model.state_dict(), "mnist_cnn.pt")

[LOGG@main] type(dataset1)=<class 'torchvision.datasets.mnist.MNIST'>, dataset1.train_data.shape=torch.Size([60000, 28, 28]), dataset1.train_labels.shape=torch.Size([60000]); dataset1.test_data.shape=torch.Size([60000, 28, 28]), dataset1.test_labels.shape=torch.Size([60000])
[LOGG@main] type(dataset2)=<class 'torchvision.datasets.mnist.MNIST'>, dataset2.train_data.shape=torch.Size([10000, 28, 28]), dataset2.train_labels.shape=torch.Size([10000]); dataset2.test_data.shape=torch.Size([10000, 28, 28]), dataset2.test_labels.shape=torch.Size([10000])


/usr/local/lib/python3.10/dist-packages/torchvision/datasets/mnist.py:76: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")
/usr/local/lib/python3.10/dist-packages/torchvision/datasets/mnist.py:66: UserWarning: train_labels has been renamed targets
  warnings.warn("train_labels has been renamed targets")
/usr/local/lib/python3.10/dist-packages/torchvision/datasets/mnist.py:81: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.10/dist-packages/torchvision/datasets/mnist.py:71: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.295299
Train Epoch: 1 [2560/60000 (4%)]	Loss: 0.629554
Train Epoch: 1 [5120/60000 (9%)]	Loss: 0.541710
Train Epoch: 1 [7680/60000 (13%)]	Loss: 0.189602
Train Epoch: 1 [10240/60000 (17%)]	Loss: 0.240388
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.151567
Train Epoch: 1 [15360/60000 (26%)]	Loss: 0.086300
Train Epoch: 1 [17920/60000 (30%)]	Loss: 0.117011
Train Epoch: 1 [20480/60000 (34%)]	Loss: 0.028013
Train Epoch: 1 [23040/60000 (38%)]	Loss: 0.266563
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.075503
Train Epoch: 1 [28160/60000 (47%)]	Loss: 0.237870
Train Epoch: 1 [30720/60000 (51%)]	Loss: 0.048076
Train Epoch: 1 [33280/60000 (55%)]	Loss: 0.127459
Train Epoch: 1 [35840/60000 (60%)]	Loss: 0.091547
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.125097
Train Epoch: 1 [40960/60000 (68%)]	Loss: 0.149741
Train Epoch: 1 [43520/60000 (72%)]	Loss: 0.291105
Train Epoch: 1 [46080/60000 (77%)]	Loss: 0.228105
Train Epoch: 1 [48640/60000 (81%)]	Loss: 0.020115
Train Epoc

KeyboardInterrupt: 

In [ ]:
# _model = Net().to(device)
